In [2]:
import pandas as pd
from pickle import dump
import numpy as np

articles = pd.read_csv('all_data.csv', delimiter = ',')

text = articles['Title'].str.cat(sep='\n')
text_size = len(text)

chars = sorted(list(set(text)))
mapping = dict((c,i) for i,c in enumerate(chars))
dump(mapping, open('mapping1.pkl', 'wb'))

vocab_size = len(mapping)
print("Vocab size: ", vocab_size)

encoded_text = [mapping[char] for char in text]
encode_size = len(encoded_text)
print('Code size:', encode_size)

seqlen = 10
batchsize = 1024
batchnum = int((encode_size - seqlen) / batchsize)

from keras.utils import to_categorical

def myGenerator():
    while 1:
        for i in range(batchnum):
            X_batch = []
            y_batch = []

            for j in range(batchsize):
                X_batch.append(encoded_text[i*batchsize+j:i*batchsize+j+seqlen])
                y_batch.append(encoded_text[i*batchsize+j+seqlen:i*batchsize+j+seqlen+1])

            X_batch = np.array([to_categorical(x, num_classes=vocab_size) for x in X_batch])
            y_batch = np.array(to_categorical(y_batch, num_classes=vocab_size))

            yield (X_batch, y_batch)


from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, SimpleRNN
from keras.models import Model

model = Sequential()
model.add(LSTM(300, return_sequences=True, input_shape=(seqlen, vocab_size)))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(75))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

my_generator = myGenerator()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(my_generator, steps_per_epoch = batchnum, epochs = 20, verbose=1)
model.save('model_3lay.h5')


FileNotFoundError: ignored